In [7]:
import pandas as pd
import re
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import requests
import time

In [8]:
# Data Collection
yelp_data_file = 'Restaurants_Yelp_With_Lat.csv'
df = pd.read_csv(yelp_data_file)

df.head()

,restaurant_name,price_range,rating,total_reviews,category,location,province,address,Latitude,Longitude,Province
0,The Flying Pig,$$,4.0,1300,Canadian (New),vancouver,BC,"1168 Hamilton Street Unit 104 Vancouver, BC V6...",49.275039,-123.122038,BC
1,Kingyo,$$,4.4,1100,Japanese,vancouver,BC,"871 Denman Street Vancouver, BC V6G 2L9 Canada",49.290612,-123.137095,BC
2,Miku,$$$,4.4,2200,Japanese,vancouver,BC,"200 Granville Street Suite 70 Vancouver, BC V6...",49.286826,-123.112583,BC
3,Kosoo,$$,4.3,88,Korean,vancouver,BC,"1128 Robson Street Vancouver, BC V6E 1B2 Canada",49.284988,-123.125959,BC
4,Dinesty Dumpling House,$$,3.9,928,Chinese,vancouver,BC,"1719 Robson Street Vancouver, BC V6G 1C8 Canada",49.290780,-123.134073,BC


In [9]:
print(df.columns)

Index(['restaurant_name', 'price_range', 'rating', 'total_reviews', 'category',
       'location', 'province', 'address', 'Latitude', 'Longitude', 'Province'],
      dtype='object')


In [10]:
len(df)

1157

In [12]:
    def get_lat_long_from_postal_code(postal_code):
        geocoding_url = f"https://nominatim.openstreetmap.org/search"
        params = {"q": postal_code, "format": "json", "limit": 1}

        retries = 5
        for i in range(retries):
            try:
                response = requests.get(geocoding_url, params=params)
                response.raise_for_status()  # Raise an exception for non-2xx status codes
                data = response.json()
                if data:
                    latitude = float(data[0]['lat'])
                    longitude = float(data[0]['lon'])
                    return latitude, longitude
                else:
                    raise ValueError("Invalid postal code or location not found.")
            except requests.exceptions.RequestException as e:
                print(f"Error occurred while geocoding: {e}")
                if i < retries - 1:
                    print("Retrying in 5 seconds...")
                    time.sleep(5)
                else:
                    raise

    def suggest_restaurants(postal_code, max_distance_km=5, top_n=5):
        # Convert postal code to Latitude and Longitude
        user_latitude, user_longitude = get_lat_long_from_postal_code(postal_code)

        # Calculate distances between user location and each restaurant
        df['Distance'] = df.apply(
            lambda row: geodesic((user_latitude, user_longitude), (row['Latitude'], row['Longitude'])).kilometers,
            axis=1
        )

        # Filter restaurants within the specified maximum distance
        nearby_restaurants = df[df['Distance'] <= max_distance_km]

        # Sort restaurants based on rating and total_reviews (you can customize this)
        sorted_restaurants = nearby_restaurants.sort_values(by=['rating', 'total_reviews'], ascending=False)

        # Get the top 'top_n' restaurant suggestions
        restaurant_suggestions = sorted_restaurants.head(top_n)

        return restaurant_suggestions[['restaurant_name', 'category', 'rating', 'total_reviews']]

    # Sample usage:
    postal_code = 'V6E 1B2'
    suggested_restaurants = suggest_restaurants(postal_code)
    print(suggested_restaurants)


        restaurant_name       category  rating  total_reviews
44            Tom Sushi     Sushi Bars     4.7            194
65    The Red Accordion  Cocktail Bars     4.7             72
103     Absinthe Bistro         French     4.6            137
17   Happy Noodle House        Dim Sum     4.6             81
79         Burger Crush        Burgers     4.6             19
